In [ ]:
#import required libraries
import tensorflow as tf
import csv
import random
import numpy as np
import pandas as pd
import io
import gzip

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install fasttext
! pip install fasttext

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093765 sha256=e6195c0609b0889c82e32c6dc799a059bb7e564aaa492df0e1e1d977d524ebd8
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.bin.gz

--2021-06-08 16:26:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 481477801 (459M) [binary/octet-stream]
Saving to: ‘cc.si.300.vec.gz’

cc.si.300.vec.gz    100%[===================>] 459.17M  31.9MB/s    in 15s     

2021-06-08 16:27:12 (30.8 MB/s) - ‘cc.si.300.vec.gz’ saved [481477801/481477801]

--2021-06-08 16:27:12--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3181346570 (3.0G) [application/octet-stream]
Saving to: ‘cc.si.3

In [ ]:
!gzip -d cc.si.300.bin.gz

gzip: cc.si.300.bin.gz: No such file or directory


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!sudo python setup.py install

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
import fasttext
import fasttext.util
ft = fasttext.load_model('/content/cc.si.300.bin')
ft.get_dimension()

300

In [ ]:
import fasttext.util
fasttext.util.reduce_model(ft, 100)

100


In [ ]:
#reading the dataset
dataset = []
with open("/content/drive/MyDrive/sinhala-hate-speech-dataset.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
      label = row[2]
      post = row[1]
      item = []
      item.append(post)
      if label == '0':
        item.append(0)
      elif label == '1':
        item.append(1)
      else:
        print('ERROR') 
        continue  
      dataset.append(item)

ERROR


In [ ]:
len(dataset)

6345

In [ ]:
#separate posts and labels
posts=[]
labels=[]
for x in range(len(dataset)):
    posts.append(dataset[x][0])
    labels.append(dataset[x][1])

In [ ]:
from sklearn.model_selection import train_test_split
posts_train, posts_test, labels_train, labels_test = train_test_split(np.asarray(posts), np.asarray(labels), test_size=0.1, random_state=42)

In [ ]:
vocab_size = 10000
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(posts_train)

word_index = tokenizer.word_index
vocab_size=len(word_index)

sequences = tokenizer.texts_to_sequences(posts_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(posts_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [ ]:
embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.5)),
    tf.keras.layers.Dense(24, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 120, 100)          1760000   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 64)                34048     
_________________________________________________________________
dense_21 (Dense)             (None, 24)                1560      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 25        
Total params: 1,795,633
Trainable params: 35,633
Non-trainable params: 1,760,000
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
num_epochs=100
history = model.fit(padded, labels_train, epochs=num_epochs, batch_size = 32)

Epoch 1/100
179/179 [==============================] - 18s 80ms/step - loss: 0.6259 - accuracy: 0.7118
Epoch 2/100
179/179 [==============================] - 15s 83ms/step - loss: 0.6967 - accuracy: 0.5347
Epoch 3/100
179/179 [==============================] - 15s 82ms/step - loss: 0.5390 - accuracy: 0.7306
Epoch 4/100
179/179 [==============================] - 15s 82ms/step - loss: 0.4445 - accuracy: 0.8091
Epoch 5/100
179/179 [==============================] - 15s 83ms/step - loss: 0.4315 - accuracy: 0.8132
Epoch 6/100
179/179 [==============================] - 15s 82ms/step - loss: 0.4200 - accuracy: 0.8169
Epoch 7/100
179/179 [==============================] - 15s 83ms/step - loss: 0.3930 - accuracy: 0.8271
Epoch 8/100
179/179 [==============================] - 15s 83ms/step - loss: 0.3889 - accuracy: 0.8241
Epoch 9/100
179/179 [==============================] - 15s 82ms/step - loss: 0.3629 - accuracy: 0.8374
Epoch 10/100
179/179 [==============================] - 15s 84ms/step - l

In [ ]:
  model.evaluate(testing_padded, labels_test, batch_size = 32)

20/20 [==============================] - 1s 20ms/step - loss: 0.3360 - accuracy: 0.8677


[0.33602118492126465, 0.8677165508270264]